In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM,GRU,SimpleRNN
from keras.layers import Dense, Activation, Dropout
from keras.layers import Embedding
from keras.layers import BatchNormalization
import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping

import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff

In [2]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


In [3]:
train = pd.read_csv(r"C:\Users\Frhnx\Downloads\Spam_SMS.csv")

In [4]:
train

,Class,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5569,spam,This is the 2nd time we have tried 2 contact u...
5570,ham,Will ü b going to esplanade fr home?
5571,ham,"Pity, * was in mood for that. So...any other s..."
5572,ham,The guy did some bitching but I acted like i'd...


In [5]:
from sklearn.preprocessing import LabelEncoder
s_encoder=LabelEncoder()
train['Class']=s_encoder.fit_transform(train['Class'])

In [6]:
train

,Class,Message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5569,1,This is the 2nd time we have tried 2 contact u...
5570,0,Will ü b going to esplanade fr home?
5571,0,"Pity, * was in mood for that. So...any other s..."
5572,0,The guy did some bitching but I acted like i'd...


In [8]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train.Message, train.Class.values, 
                                                  stratify=train.Class.values, 
                                                  random_state=42, 
                                                  test_size=0.2, shuffle=True)

In [9]:
# using keras tokenizer here
token = Tokenizer(num_words=None)
max_len = 1500

token.fit_on_texts(list(xtrain) + list(xvalid))
xtrain_seq = token.texts_to_sequences(xtrain)
xvalid_seq = token.texts_to_sequences(xvalid)

#zero pad the sequences
xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = sequence.pad_sequences(xvalid_seq, maxlen=max_len)

word_index = token.word_index

In [10]:
# load the GloVe vectors in a dictionary:

embeddings_index = {}
f = open(r"C:\Users\Frhnx\Downloads\glove.42B.300d\glove.42B.300d.txt",'r',encoding='utf-8')
for line in tqdm(f):
    values = line.split(' ')
    word = values[0]
    coefs = np.asarray([float(val) for val in values[1:]])
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

1917495it [02:03, 15503.47it/s]

Found 1917495 word vectors.


In [13]:
# create an embedding matrix for the words we have in the dataset
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

100%|██████████| 9004/9004 [00:00<00:00, 398248.57it/s]


In [15]:
%%time
with strategy.scope():
    
    # A simple LSTM with glove embeddings and one dense layer
    model = Sequential()
    model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))

    model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.3))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
    
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │     2,701,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,701,500 (10.31 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 2,701,500 (10.31 MB)

CPU times: total: 0 ns
Wall time: 28.9 ms


In [17]:
model.fit(xtrain_pad, ytrain, epochs=15, batch_size=64*strategy.num_replicas_in_sync,validation_split=0.1)

Epoch 1/15
63/63 ━━━━━━━━━━━━━━━━━━━━ 244s 4s/step - accuracy: 0.8639 - loss: 0.3260 - val_accuracy: 0.9529 - val_loss: 0.1416
Epoch 2/15
63/63 ━━━━━━━━━━━━━━━━━━━━ 260s 4s/step - accuracy: 0.9749 - loss: 0.0738 - val_accuracy: 0.9574 - val_loss: 0.1145
Epoch 3/15
63/63 ━━━━━━━━━━━━━━━━━━━━ 268s 4s/step - accuracy: 0.9818 - loss: 0.0552 - val_accuracy: 0.9596 - val_loss: 0.0994
Epoch 4/15
63/63 ━━━━━━━━━━━━━━━━━━━━ 263s 4s/step - accuracy: 0.9846 - loss: 0.0540 - val_accuracy: 0.9686 - val_loss: 0.0792
Epoch 5/15
63/63 ━━━━━━━━━━━━━━━━━━━━ 263s 4s/step - accuracy: 0.9871 - loss: 0.0456 - val_accuracy: 0.9686 - val_loss: 0.0748
Epoch 6/15
63/63 ━━━━━━━━━━━━━━━━━━━━ 265s 4s/step - accuracy: 0.9888 - loss: 0.0302 - val_accuracy: 0.9731 - val_loss: 0.0747
Epoch 7/15
63/63 ━━━━━━━━━━━━━━━━━━━━ 267s 4s/step - accuracy: 0.9902 - loss: 0.0335 - val_accuracy: 0.9731 - val_loss: 0.0781
Epoch 8/15
63/63 ━━━━━━━━━━━━━━━━━━━━ 263s 4s/step - accuracy: 0.9925 - loss: 0.0228 - val_accuracy: 0.9709 - v